In [1]:
import torch 
import numpy as np
from torch.autograd import Variable
from torch import nn
from torch.optim import SGD

In [2]:
# # affinity regression:

# # RWA^T = Y
# # R:  low-level embeddings
# # W: weight matrix to learn
# # A: RNA/protein sequences to match
# # Y: known matches
# (samples, embedding dim)     (hidden1, hidden2)   (rna_samples, dim_rna_samples)^T      = (# RNAs, expression over dim_rna_samples)
# (100, 1000)                  (1000, 45)            (45, 20394)                          = (100, 20394)
    
    
# emb_dim: hidden embedding
# match_dim: # rna_samples
# out_dim: d

# samples = batch_size
# embedding_dim = learned embedding dim (emb_dim)
# hidden1 = embedding_dim
# hidden2 = dimension of rna samples (rna_dim)
# rna_samples = # of RNA samples we're looking at

# output dimensions:
#     # samples we're looking at
#     # number of RNA samples we're looking at 

class AfinityRegression(nn.Module):
    def __init__(self, emb_dim, rna_dim):
        super(AfinityRegression, self).__init__()
        # want to learn weights after applying to embedding layer (First do RW , then RW A^T)
        self.lin1 = nn.Linear(emb_dim, rna_dim)
    
    def forward(self, batch_size, embs, rna_samples):
        self.batch_size = batch_size
        x = self.lin1(embs)
        print(x.size())
        print(rna_samples.size())
        x = torch.matmul(x, rna_samples)

        return x





In [3]:

# samples = batch_size                              777
# embedding_dim = learned embedding dim (emb_dim)   10
# hidden1 = embedding_dim                           10
# hidden2 = dimension of rna samples (rna_dim)      2555
# rna_samples = # of RNA samples we're looking at   30

test_model = AfinityRegression(emb_dim=10, rna_dim=2555)
optimizer = SGD(test_model.parameters(), lr = 0.001)


learned_embs = Variable(torch.randn((777,10)))
poss_matches = Variable(torch.randn((2555, 30)))
known_matches = Variable(torch.randn((777,30)))

outs = test_model.forward(3, learned_embs, poss_matches)
new_mat = known_matches - outs
losses = torch.bmm(new_mat.view(new_mat.size()[0], 1, new_mat.size()[1]),
                   new_mat.view(new_mat.size()[0], new_mat.size()[1], 1)
                  )
print(losses.size())
torch.sqrt(losses).size()


torch.Size([777, 2555])
torch.Size([2555, 30])
torch.Size([777, 1, 1])


torch.Size([777, 1, 1])

In [5]:
outs.size()

torch.Size([777, 30])